In [1]:
import pandas as pd
import numpy as np

In [2]:
invoice_data=pd.read_csv('C:\\Users\\Fakhruddin\\Capstone Project\\Mahindra First Choice Dataset\\Final_invoice.csv')

C:\Users\Fakhruddin\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (9,10,14,15,21,41,51) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
invoice_data.columns

Index(['Unnamed: 0', 'Amt Rcvd From Custom', 'Amt Rcvd From Ins Co',
       'Area / Locality', 'CGST(14%)', 'CGST(2.5%)', 'CGST(6%)', 'CGST(9%)',
       'CITY', 'Cash /Cashless Type', 'Claim No.', 'Cust Type', 'Customer No.',
       'District', 'Expiry Date', 'Gate Pass Date', 'Gate Pass Time',
       'IGST(12%)', 'IGST(18%)', 'IGST(28%)', 'IGST(5%)', 'Insurance Company',
       'Invoice Date', 'Invoice No', 'Invoice Time', 'Job Card No',
       'JobCard Date', 'JobCard Time', 'KMs Reading', 'Labour Total', 'Make',
       'Misc Total', 'Model', 'ODN No.', 'OSL Total', 'Order Type',
       'Outstanding Amt', 'Parts Total', 'Pin code', 'Plant', 'Plant Name1',
       'Policy no.', 'Print Status', 'Recovrbl Exp', 'Regn No',
       'SGST/UGST(14%)', 'SGST/UGST(2.5%)', 'SGST/UGST(6%)', 'SGST/UGST(9%)',
       'Service Advisor Name', 'TDS amount', 'Technician Name',
       'Total Amt Wtd Tax.', 'Total CGST', 'Total GST', 'Total IGST',
       'Total SGST/UGST', 'Total Value', 'User ID'],
     

In [5]:
selected_df=invoice_data[['District','Order Type','JobCard Date']]
selected_df['JobCard Date']=pd.to_datetime(selected_df['JobCard Date'])
selected_df=selected_df.rename(columns={'District':'State'})
selected_df.head()

C:\Users\Fakhruddin\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,State,Order Type,JobCard Date
0,Maharashtra,Paid Service,2014-12-13
1,Maharashtra,SMC Value Package,2015-01-02
2,Maharashtra,Running Repairs,2015-01-02
3,Maharashtra,SMC Redemption,2015-01-03
4,Maharashtra,Paid Service,2015-01-02


In [6]:
df=selected_df.groupby(['State','JobCard Date'])['Order Type'].value_counts().to_frame()
df.head()

Order Type
State                JobCard Date Order Type                 
Andaman and Nico.In. 2015-09-30   Running Repairs           1
                     2015-12-15   Accidental                1
                     2015-12-17   Running Repairs           1
                     2015-12-31   Running Repairs           1
                     2016-07-16   Running Repairs           1

In [7]:
df=df.rename(columns={'Order Type':'Count'})
df=df.reset_index()
df.head()

,State,JobCard Date,Order Type,Count
0,Andaman and Nico.In.,2015-09-30,Running Repairs,1
1,Andaman and Nico.In.,2015-12-15,Accidental,1
2,Andaman and Nico.In.,2015-12-17,Running Repairs,1
3,Andaman and Nico.In.,2015-12-31,Running Repairs,1
4,Andaman and Nico.In.,2016-07-16,Running Repairs,1


In [60]:
df[df['State']=='Goa'].shape

(23, 4)

In [8]:
from datetime import datetime as datetime
import dash
import dash_html_components as html
import dash_core_components as dcc
import plotly
from dash.dependencies import Input, Output, State
import plotly.graph_objects as go

In [42]:
traces=[]
            
trace1= go.Scatter(
                    x = filtered_df['JobCard Date'],
                    y = filtered_df['count'],
                    name = selected_state,
                  #  mode = 'markers',
                    mode='markers',
                    #marker = {
                        #'size': 12,
                       # 'color': 'rgb(51,204,153)',
                       #'symbol': 'pentagon',
                      #  'line': {'width': 2},
                    text=filtered_df['Order Type']
                        
                )
traces.append(trace1)

NameError: name 'filtered_df' is not defined

In [40]:
app = dash.Dash()
app.layout = html.Div([
    html.H1('State Order type time analysis'),
    html.Div([
        html.H3('Select State:', style={'paddingRight':'30px'}),
        # replace dcc.Input with dcc.Options, set options=options
        dcc.Dropdown(
            id='my_state',
            options=[{'label':i,'value':i} for i in df.State.unique()],
            value='Maharashtra',
            multi=False
        )
        ], style={'display':'inline-block', 'verticalAlign':'top', 'width':'30%'}),
      html.Div([
        html.H3('Select start and end dates:'),
        dcc.DatePickerRange(
            id='my_date_picker',
            min_date_allowed=df['JobCard Date'].min(),
            max_date_allowed=df['JobCard Date'].max(),
            start_date=datetime(2012,5,1),
            end_date=datetime(2012,6,1)
        )
    ], style={'display':'inline-block'}),
     html.Div([
        html.Button(
            id='submit-button',
            n_clicks=0,
            children='Submit',
            style={'fontSize':24, 'marginLeft':'30px'}
        ),
    ], style={'display':'inline-block'}),
     dcc.Graph(
        id='my_graph'
    )
    
])
#######################################################################################
@app.callback(
    Output('my_graph', 'figure'),
    [Input('submit-button', 'n_clicks')],
    [State('my_state', 'value'),
    State('my_date_picker', 'start_date'),
    State('my_date_picker', 'end_date')])

#######################################################################################
def update_graph(n_clicks,selected_state,start_date,end_date):
    start = datetime.strptime(start_date[:10], '%Y-%m-%d')
    end = datetime.strptime(end_date[:10], '%Y-%m-%d')
    filtered_df=df[(df['State']==selected_state) &
                   (df['JobCard Date']>=start)&(df['JobCard Date']<=end)
                  ]
    traces=[]
            
    trace1= go.Scatter(
                    x = filtered_df['JobCard Date'],
                    y = filtered_df['count'],
                    name = selected_state,
                  #  mode = 'markers',
                    mode='markers',
                    #marker = {
                        #'size': 12,
                       # 'color': 'rgb(51,204,153)',
                       #'symbol': 'pentagon',
                      #  'line': {'width': 2},
                    text=filtered_df['Order Type']
                        
                )
    traces.append(trace1)
    return {
        'data': traces,
        'layout': go.Layout(
            title='Revenue Generated in a State on the basis of Car Brands ',
            xaxis={ 'title': 'Car Brands'},
            yaxis={'title': 'Revenue'},
            hovermode='closest')
        
    

        } 
if __name__ == '__main__':
    app.run_server()    

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 in production, use a production WSGI server like gunicorn instead.

 in production, use a production WSGI server like gunicorn instead.

 in production, use a production WSGI server like gunicorn instead.

 in production, use a production WSGI server like gunicorn instead.

 in production, use a production WSGI server like gunicorn instead.

 in production, use a production WSGI server like gunicorn instead.

 in production, use a production WSGI server like gunicorn instead.

 

 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [27/Jun/2020 15:18:17] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [27/Jun/2020 15:18:18] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [27/Jun/2020 15:18:18] "GET /_dash-dependencies HTTP/1.1" 200 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\Fakhruddin\Anaconda3\lib\site-packages\pandas\core\indexes\base.py", line 2646, in get_loc
    return self._engine.get_loc(key)
  File "pandas\_libs\index.pyx", line 111, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\index.pyx", line 138, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\hashtable_class_helper.pxi", line 1619, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas\_libs\hashtable_class_helper.pxi", line 1627, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: 'count'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Fakhruddin\Anaconda3\lib\site-packages\flask\app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\Fakhruddin\Anaconda3\lib\site-packages\flask\app.py", line 1951, in full_dispatch_request
    rv = s

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\Fakhruddin\Anaconda3\lib\site-packages\pandas\core\indexes\base.py", line 2646, in get_loc
    return self._engine.get_loc(key)
  File "pandas\_libs\index.pyx", line 111, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\index.pyx", line 138, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\hashtable_class_helper.pxi", line 1619, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas\_libs\hashtable_class_helper.pxi", line 1627, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: 'count'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Fakhruddin\Anaconda3\lib\site-packages\flask\app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\Fakhruddin\Anaconda3\lib\site-packages\flask\app.py", line 1951, in full_dispatch_request
    rv = s

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\Fakhruddin\Anaconda3\lib\site-packages\pandas\core\indexes\base.py", line 2646, in get_loc
    return self._engine.get_loc(key)
  File "pandas\_libs\index.pyx", line 111, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\index.pyx", line 138, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\hashtable_class_helper.pxi", line 1619, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas\_libs\hashtable_class_helper.pxi", line 1627, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: 'count'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Fakhruddin\Anaconda3\lib\site-packages\flask\app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\Fakhruddin\Anaconda3\lib\site-packages\flask\app.py", line 1951, in full_dispatch_request
    rv = s

...

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\Fakhruddin\Anaconda3\lib\site-packages\pandas\core\indexes\base.py", line 2646, in get_loc
    return self._engine.get_loc(key)
  File "pandas\_libs\index.pyx", line 111, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\index.pyx", line 138, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\hashtable_class_helper.pxi", line 1619, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas\_libs\hashtable_class_helper.pxi", line 1627, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: 'count'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Fakhruddin\Anaconda3\lib\site-packages\flask\app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\Fakhruddin\Anaconda3\lib\site-packages\flask\app.py", line 1951, in full_dispatch_request
    rv = s

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\Fakhruddin\Anaconda3\lib\site-packages\pandas\core\indexes\base.py", line 2646, in get_loc
    return self._engine.get_loc(key)
  File "pandas\_libs\index.pyx", line 111, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\index.pyx", line 138, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\hashtable_class_helper.pxi", line 1619, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas\_libs\hashtable_class_helper.pxi", line 1627, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: 'count'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Fakhruddin\Anaconda3\lib\site-packages\flask\app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\Fakhruddin\Anaconda3\lib\site-packages\flask\app.py", line 1951, in full_dispatch_request
    rv = s

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\Fakhruddin\Anaconda3\lib\site-packages\pandas\core\indexes\base.py", line 2646, in get_loc
    return self._engine.get_loc(key)
  File "pandas\_libs\index.pyx", line 111, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\index.pyx", line 138, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\hashtable_class_helper.pxi", line 1619, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas\_libs\hashtable_class_helper.pxi", line 1627, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: 'count'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Fakhruddin\Anaconda3\lib\site-packages\flask\app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\Fakhruddin\Anaconda3\lib\site-packages\flask\app.py", line 1951, in full_dispatch_request
    rv = s

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\Fakhruddin\Anaconda3\lib\site-packages\pandas\core\indexes\base.py", line 2646, in get_loc
    return self._engine.get_loc(key)
  File "pandas\_libs\index.pyx", line 111, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\index.pyx", line 138, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\hashtable_class_helper.pxi", line 1619, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas\_libs\hashtable_class_helper.pxi", line 1627, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: 'count'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Fakhruddin\Anaconda3\lib\site-packages\flask\app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\Fakhruddin\Anaconda3\lib\site-packages\flask\app.py", line 1951, in full_dispatch_request
    rv = s

127.0.0.1 - - [27/Jun/2020 15:18:24] "POST /_dash-update-component HTTP/1.1" 500 -


In [58]:
app = dash.Dash(__name__)
######################################################################################################
#Layout of the dashboard:
app.layout= html.Div([
#####################################################################################################
      html.Div([
        html.Label(['Based on Make State revenue Analysis']),
        dcc.Dropdown(
            id='my_state',
            options=[{'label':i,'value':i} for i in df.State.unique()],
            value='Maharashtra',
            multi=False,
            clearable=False,
            style={'width':'50%'}
        ),
    ]),

    html.Div([
        dcc.Graph(id='State_Make_revenue_graph',
                 style={'width':'70%'}
                 )
    ])
###################################################################################    
    ])
@app.callback(Output('State_Make_revenue_graph', 'figure'),
              [Input('my_state','value')])
def update_figure(selected_state):
    filtered_df = df[df['State'] == selected_state]
    traces=[]
    #for selected_state in df1.State.unique():
    
    trace1 = go.Line(
             x = filtered_df['JobCard Date'],
             y=filtered_df['Count'],   
            name = selected_state,
            text=df['Order Type'],
            mode='lines+markers')
            #marker = dict(color = 'rgba(239,222,205, 0.7)',
            #line=dict(color='rgb(0,0,0)',width=1.5)))
    traces.append(trace1)
    
    

    return {
        'data': traces,
        'layout': go.Layout(
            title='Revenue Generated in a State on the basis of Car Brands ',
            xaxis={ 'title': 'Car Brands'},
            yaxis={'title': 'Revenue'},
            hovermode='closest')
        
    

        }
if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 in product

 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [27/Jun/2020 18:08:54] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [27/Jun/2020 18:08:54] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [27/Jun/2020 18:08:54] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [27/Jun/2020 18:08:54] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [27/Jun/2020 18:09:20] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [27/Jun/2020 18:09:31] "POST /_dash-update-component HTTP/1.1" 200 -
